In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
pip install datasets

In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate
import torch
from datasets import load_dataset

# Define and load the tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("lakshyakh93/deberta_finetuned_pii")
# Load model
model = AutoModelForTokenClassification.from_pretrained("lakshyakh93/deberta_finetuned_pii")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.31k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/555M [00:00<?, ?B/s]

In [ ]:
train_set = load_dataset("ai4privacy/pii-masking-300k", split='train')
dataset = train_set.shuffle().select(range(20))
print(dataset)

# Define label mapping
label_map = {"O": 0, "TIME": 1, "DATE": 2, "LASTNAME1": 3, "LASTNAME2": 4, "EMAIL": 5, "SOCIALNUMBER": 6}

# Function to tokenize and align labels
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    labels = ["O"] * len(tokenized_inputs["input_ids"])

    # Adjust indexing to fit the tokenization
    example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length

    for span in example["privacy_mask"]:
        start, end, label_name = span["start"], span["end"], span["label"]
        label_id = label_map.get(label_name, 0)

        span_tokens = tokenizer(example["source_text"][start:end], add_special_tokens=False).tokens()
        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs["input_ids"]) - len(numeric_labels))

    tokenized_inputs["labels"] = numeric_labels
    return tokenized_inputs


# Apply function to dataset
encoded_dataset = dataset.map(tokenize_and_align_labels, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(encoded_dataset["input_ids"])
attention_mask = torch.tensor(encoded_dataset["attention_mask"])
labels = torch.tensor(encoded_dataset["labels"])

# Define metric computation
metric = evaluate.load("accuracy")  # You may want to load relevant metrics for token classification

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)  # Note: axis=2 for token classification
    true_labels = labels != -100  # Masking out unnecessary labels

    # Flatten predictions and true_labels to compute accuracy
    flattened_predictions = predictions[true_labels]
    flattened_labels = labels[true_labels]

    results = metric.compute(references=flattened_labels, predictions=flattened_predictions)
    return results


# Tokenize validation set
val_set = load_dataset("ai4privacy/pii-masking-300k", split='validation')  # Ensure you have a validation set
small_val = val_set.shuffle().select(range(20))
encoded_small_val = small_val.map(tokenize_and_align_labels, batched=False)

# Training arguments
train_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=10,
    eval_steps=10,
    gradient_accumulation_steps=2,  # Adjust if needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_small_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/6.31k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/555M [00:00<?, ?B/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Run training
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.253812,0.966014
2,1.595900,0.041401,0.992850
3,1.595900,0.021642,0.995234


TrainOutput(global_step=15, training_loss=1.0684983124335607, metrics={'train_runtime': 38.0008, 'train_samples_per_second': 1.579, 'train_steps_per_second': 0.395, 'total_flos': 18303156633600.0, 'train_loss': 1.0684983124335607, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.021642297506332397,
 'eval_accuracy': 0.9952336545435706,
 'eval_runtime': 1.0887,
 'eval_samples_per_second': 18.371,
 'eval_steps_per_second': 2.756,
 'epoch': 3.0}

In [5]:
from transformers import pipeline

# Initialize a token classification pipeline
pii_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Function to apply PII masking to a specific text column
def redact_column(dataset, column_name):
    redacted_texts = []

    for text in dataset[column_name]:
        # Get model predictions for the text
        predictions = pii_pipeline(text)

        # Replace PII spans with placeholders
        redacted_text = text
        for entity in sorted(predictions, key=lambda x: x['start'], reverse=True):  # Sort in reverse to avoid shifting positions
            label = entity['entity_group']
            redacted_text = redacted_text[:entity['start']] + f"[{label}]" + redacted_text[entity['end']:]

        redacted_texts.append(redacted_text)

    return redacted_texts

In [20]:
df = pd.DataFrame({
    'source_text': [
        'Her name is Jane and her phone number is 805-847-4340.',
        'Please reach out to John Smith at john.smith@example.com for more details.',
        'His credit card number is 1234-5678-9012-3456, the routing number is 834245680, and it expires on 11/25.',
        'You can contact Mary at 555-123-4567 or at her office in 123 Elm Street, Denver, CO 62704.',
        'The passport number is K12345678 issued by the US Department of State.',
        "Robert's social security number is 987-65-4320, which needs to be redacted.",
        "The user's login attempt was from IP address 192.168.0.1.",
        "Patient Sarah Taylor's medical record indicates a diagnosis of hypertension, and her insurance ID is ABCD123456.",
        "I live in Denver, CO."
    ]
})

redact_column(df, 'source_text')

['Her name is[FIRSTNAME] and her phone number is[PHONE_NUMBER][PHONE_NUMBER].',
 'Please reach out to[FULLNAME] at[EMAIL] for more details.',
 'His credit card number is[CREDITCARDNUMBER][CREDITCARDNUMBER], the routing number is[PIN][PIN], and it expires on[DATE].',
 'You can contact[FIRSTNAME] at[PHONE_NUMBER] or at her office in[STREETADDRESS],[STATE],[STATE][ZIPCODE][ZIPCODE].',
 'The passport number is[PIN][PIN][PIN] issued by the US Department of State.',
 "[FIRSTNAME]'s social security number is[SSN][SSN], which needs to be redacted.",
 "The user's login attempt was from IP address[IPV4][IP].",
 "Patient[FIRSTNAME][MIDDLENAME]'s medical record indicates a diagnosis of hypertension, and her insurance ID is[PIN][PIN].",
 'I live in[CITY],[STATE].']

In [12]:
# Use the model directly
inputs = tokenizer(
    "Her name is Jane and her phone number is 805-847-4340", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss

In [13]:
# Tokenize the input text to get tokenized words for each label
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# # Print each token with its predicted label
# for token, label in zip(tokens, predicted_tokens_classes):
#     print(f"{token}: {label}")

# Reconstruct the sentence with labels
labeled_sentence = []
for token, label in zip(tokens, predicted_tokens_classes):
    if not token.startswith("##"):  # Handle subword tokens
        labeled_sentence.append(f"{token}({label})")
    else:
        # Append subword tokens without a space
        labeled_sentence[-1] = labeled_sentence[-1][:-len(label)-2] + token[2:] + f"({label})"

# Join the labeled sentence into a single string
result_sentence = " ".join(labeled_sentence)

# Print the result
print(result_sentence)

ĠHer(O) Ġname(O) Ġis(O) ĠJane(O) Ġand(O) Ġher(O) Ġphone(O) Ġnumber(O) Ġis(O) Ġ8(I-IPV6) 05(I-IPV6) -(I-IPV6) 8(I-IPV6) 47(I-IPV6) -(I-IPV6) 43(I-IPV6) 40(I-IPV6)


In [16]:
# Use the model directly
inputs = tokenizer(
    "Her name is Jane and her phone number is 512-247-6993", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Map token class IDs to labels
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

# Tokenize the input text to get the words for each token
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Reconstruct the original input sentence
original_sentence = "Her name is Jane and her phone number is 512-247-6993"  # This is the input

# Create the label sentence (space-separated labels for each token)
label_sentence = []
for token, label in zip(tokens, predicted_tokens_classes):
    if not token.startswith("##"):  # Handle subword tokens
        label_sentence.append(label)
    else:
        # Subword tokens don't affect labels but ensure they follow token merging rules
        continue

label_sentence = " ".join(label_sentence)

# Print both sentences
print(f"original sentence: {original_sentence}")
print(f"label sentence: {label_sentence}")

Original Sentence: Her name is Jane and her phone number is 512-247-6993
Label Sentence: O O O O O O O O O O I-IPV6 I-IPV6 I-IPV6 I-IPV6 I-IPV6
